In [12]:
import pandas as pd
from datetime import datetime, timedelta
from docx import Document
from docx.oxml.ns import qn
from docx.oxml import OxmlElement
from docx.shared import Inches, Pt
from docx.enum.text import WD_ALIGN_PARAGRAPH

def set_cell_borders(cell, border_color="000000", border_width=4):
    """Set the borders for a cell."""
    tc_pr = cell._element.xpath('.//w:tcPr')
    if not tc_pr:
        tc_pr = OxmlElement('w:tcPr')
        cell._element.insert(0, tc_pr)
    else:
        tc_pr = tc_pr[0]
    
    borders = OxmlElement('w:tcBorders')
    border_attrs = {
        'w:top': border_color,
        'w:left': border_color,
        'w:bottom': border_color,
        'w:right': border_color,
        'w:insideH': border_color,
        'w:insideV': border_color
    }
    
    for border in border_attrs:
        border_elem = OxmlElement(border)
        border_elem.set(qn('w:val'), 'single')
        border_elem.set(qn('w:sz'), str(border_width))  # Border width (1/8 pt)
        border_elem.set(qn('w:space'), '0')
        borders.append(border_elem)

    tc_pr.append(borders)

def set_cell_fill(cell, color):
    """Set the fill color for a cell."""
    shading = OxmlElement('w:shd')
    shading.set(qn('w:fill'), color)
    cell._element.get_or_add_tcPr().append(shading)

def set_paragraph_spacing(cell, spacing_before=0, spacing_after=0):
    """Set paragraph spacing within a cell."""
    for para in cell.paragraphs:
        para_format = para.paragraph_format
        para_format.space_after = Pt(spacing_after)
        para_format.space_before = Pt(spacing_before)
        para_format.line_spacing = Pt(12)  # Single line spacing (12 pt)

def clear_cell_text(cell):
    """Clear text from a cell and set paragraph spacing."""
    cell.text = ''
    set_paragraph_spacing(cell)  # Ensure paragraph spacing is set

def set_cell_text(cell, text, font_name='Times New Roman', font_size=11, bold=True, alignment='left'):
    """Set text and formatting in a cell."""
    cell.text = text
    
    if alignment == 'left':
        align = WD_ALIGN_PARAGRAPH.LEFT
    elif alignment == 'center':
        align = WD_ALIGN_PARAGRAPH.CENTER
    elif alignment == 'right':
        align = WD_ALIGN_PARAGRAPH.RIGHT
    else:
        raise ValueError("Invalid alignment value. Choose from 'left', 'center', or 'right'.")

    for para in cell.paragraphs:
        para_format = para.paragraph_format
        para_format.alignment = align
        para_format.space_after = Pt(0)
        
        for run in para.runs:
            run.font.name = font_name
            run.font.size = Pt(font_size)
            run.font.bold = bold
        
        para.style.font.name = font_name

# Table Manipulation Functions

def fill_row_with_cells(table, row_idx, fill_color="FFFFFF"):
    """Fill a row with cells, setting borders, fill color, and clearing text."""
    for col_idx in range(2):  # Two columns
        cell = table.cell(row_idx, col_idx)
        set_cell_borders(cell, border_color="000000", border_width=4)
        set_cell_fill(cell, color=fill_color)
        clear_cell_text(cell)

def handle_alternating_rows(table, start_row, end_row, color):
    """Handle alternating row colors."""
    for row_idx in range(start_row, end_row + 1):
        fill_row_with_cells(table, row_idx, color)

def add_merged_row(table, row_idx, fill_color="D0D0D0"):
    """Add a merged row with a specific fill color."""
    cell_1 = table.cell(row_idx, 0)
    cell_2 = table.cell(row_idx, 1)
    cell_1.merge(cell_2)
    set_cell_borders(cell_1, border_color="000000", border_width=4)
    set_cell_fill(cell_1, color=fill_color)
    clear_cell_text(cell_1)

# Document Creation Function

def create_table_with_content(doc):
    """Create a table with specified content and formatting."""
    # Set document margins
    section = doc.sections[0]
    section.left_margin = Inches(1)
    section.right_margin = Inches(1)

    # Add a table with 35 rows and 2 columns
    table = doc.add_table(rows=35, cols=2)

    # Row 0
    cell_1 = table.cell(0, 0)
    cell_2 = table.cell(0, 1)
    
    # Set borders and fill color for both cells
    set_cell_borders(cell_1, border_color="000000", border_width=4)
    set_cell_borders(cell_2, border_color="000000", border_width=4)
    set_cell_fill(cell_1, color="D0D0D0")
    set_cell_fill(cell_2, color="D0D0D0")
    
    # Set text for cell 1
    set_cell_text(cell_1, "SALE COMPARABLE\nPRG Reference: ", font_name='Times New Roman', font_size=11, bold=True, alignment='left')
    
    # Adjust paragraph spacing after PRG Reference
    for para in cell_1.paragraphs:
        if "PRG Reference" in para.text:
            para_format = para.paragraph_format
            para_format.space_after = Pt(6)  # Add 6pt space after PRG Reference line

    # Add the image to cell 2
    # The path to the image file (ensure the path is correct)
    image_path = 'PRG Logo.jpg'
    # Add the image to the cell
    paragraph = cell_2.add_paragraph()
    run = paragraph.add_run()
    run.add_picture(image_path, width=Inches(1.5))  # Adjust width as needed
    
    # Align the paragraph to the right
    paragraph.alignment = WD_ALIGN_PARAGRAPH.RIGHT
    
    # Set paragraph spacing to zero
    para_format = paragraph.paragraph_format
    para_format.space_before = Pt(0)  # Ensure no space before the paragraph
    para_format.space_after = Pt(0)   # Ensure no space after the paragraph
    para_format.line_spacing = Pt(0)  # Ensure no extra line spacing

    # Set cell padding to zero
    cell_2_pr = cell_2._element.get_or_add_tcPr()
    cell_mar = OxmlElement('w:cellMar')
    cell_mar.set(qn('w:top'), '0')
    cell_mar.set(qn('w:left'), '0')
    cell_mar.set(qn('w:bottom'), '0')
    cell_mar.set(qn('w:right'), '0')
    cell_2_pr.append(cell_mar)

    # Save the document
    doc.save('test.docx')

# Create a new Document
doc = Document()
create_table_with_content(doc)
